In [3]:
from tokenizer import Tokenizer 

tokenizer = Tokenizer.from_files(vocab_path="vocab.json",merges_path="merges.txt",special_tokens=["<|endoftext|>"])

In [5]:
type(tokenizer.vocab)

dict

In [6]:
len(tokenizer.vocab)

1001

In [ ]:
import torch.nn as nn
import torch
class My_Embedding(nn.Module):
    def __init__(self,vocab_size,embedding_dim=10):
        super().__init__()
        
        
        self.weight =nn.Parameter(torch.empty(vocab_size,embedding_dim))
        
        nn.init.trunc_normal_(self.weight,mean=0,std=0.02,a=-2,b=2)
        
    def forward(self,x):
        return self.weight[x]
    

class Model(nn.Module):
    def __init__(self,vocab_size,embedding_dim=10):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        
    def forward(self,x):
        return self.embedding(x)
    
    
embedding = Model(vocab_size=1001,embedding_dim=10)

input = "hello world"

input_ids = tokenizer.encode(input)

input_ids = torch.tensor(input_ids)

input = embedding(input_ids)

input

sim = input[0]@input[1]
sim



tensor(0.3391, grad_fn=<DotBackward0>)

In [4]:
import torch
mask = torch.tril(torch.ones(3,3,))


In [5]:
mask

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

# cs336 

### Import Packages

In [6]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from einops import rearrange
import math

## Implement Embedding class

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F


class Embedding(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, device=None, dtype=None):
        super().__init__()
        
        factory_kwargs ={'device':device,
                          'dtype':dtype}
        
        self.weight = nn.Parameter(
            torch.empty(vocab_size,d_model,**factory_kwargs)
        )
        
        std = 1
        nn.init.trunc_normal_(self.weight, mean=0, std=std, a=-3, b=3)
        
    def forward(self,x):
        
        return self.weight[x]
    

## Implement Liear Layer

In [ ]:
class Linear(nn.Module):
    def __init__(self, input_features: int, output_features: int, device=None, dtype=None):
        super().__init__()
        
        factory_kwargs = {'device':device, 'dtype':dtype}
        
        self.weight = nn.Parameter(
            torch.empty(output_features,input_features,**factory_kwargs)
        )
        
        std = (2 / (input_features + output_features)) ** 0.5
        
        nn.init.trunc_normal_(
            self.weight,
            mean=0,
            std=std,
            a= -3 *std,
            b= 3*std
        )
        
    def forward(self,x):
        
        ## input shape of x (B,seq,in) -> (b,seq,out)\
            
        output = torch.einsum(
            "b s i, o i->b s o",
            x,self.weight
        )
        
        return output

## MHA and scaled_dot_product

In [ ]:
class MyMuitiHeadAttention(nn.Module):
    def __init__(self, num_head: int, d_model:int, device=None, dtype=None):
        super().__init__()
        
        assert d_model % num_head == 0
        
        self.num_head = num_head
        self.d_model = d_model
        self.device = device
        self.dtype = dtype
        
        self.d_head = self.d_model // self.num_head
        
        self.proj_q = Linear(input_features=d_model,output_features=d_model,device=device,dtype=dtype)
        self.proj_k = Linear(input_features=d_model,output_features=d_model,device=device,dtype=dtype)
        self.proj_v = Linear(input_features=d_model,output_features=d_model,device=device,dtype=dtype)
        
        self.proj_out = Linear(input_features=d_model,output_features=d_model,device=device,dtype=dtype)
        
    def forward(self,x):
        
        b,s,_ = x.shape
        
        mask = torch.tril(torch.ones(s, s, device=x.device, dtype=torch.bool))
        mask = mask.unsqueeze(0).unsqueeze(0)  # (1,1,s,s)  
        q = rearrange(
            self.proj_q(x),
            "b s (h d)->b h s d",
            h = self.num_head
            )
        k = rearrange(
            self.proj_k(x),
            "b s (h d)->b h s d",
            h = self.num_head
            )
        v = rearrange(
            self.proj_v(x),
            "b s (h d)->b h s d",
            h = self.num_head
            )
        
        attn_score = torch.einsum("...nk,...mk->...nm",q,k) / math.sqrt(self.d_head)
        
        attn_score = attn_score.masked_fill(
            mask==0,
            float('-inf')
        )
        
        prob = torch.softmax(attn_score,dim=-1)
        
        attn_score = torch.einsum("...nm,...mk->...nk",prob,v)
        
        attn_score = rearrange(attn_score,"b h s d->b s (h d)")
        
        return self.proj_out(attn_score)